## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Najran,SA,17.4924,44.1277,63.52,31,6,5.30,clear sky
1,1,Caravelas,BR,-17.7125,-39.2481,85.91,68,97,11.59,overcast clouds
2,2,Upernavik,GL,72.7868,-56.1549,8.24,93,100,3.74,overcast clouds
3,3,Togur,RU,58.3631,82.8265,30.36,95,100,15.99,light snow
4,4,Hithadhoo,MV,-0.6000,73.0833,82.11,74,25,5.39,scattered clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Hithadhoo,MV,-0.6000,73.0833,82.11,74,25,5.39,scattered clouds
5,5,Saint-Joseph,RE,-21.3667,55.6167,77.36,84,28,8.48,scattered clouds
7,7,Meulaboh,ID,4.1363,96.1285,78.24,84,99,3.42,overcast clouds
8,8,Coihaique,CL,-45.5752,-72.0662,70.41,49,40,11.50,scattered clouds
10,10,Rikitea,PF,-23.1203,-134.9692,78.12,81,100,20.78,overcast clouds
11,11,Vaini,TO,-21.2000,-175.2000,73.56,100,100,0.00,light intensity shower rain
14,14,Butaritari,KI,3.0707,172.7902,76.57,94,100,16.98,very heavy rain
15,15,Sibolga,ID,1.7427,98.7792,76.21,91,100,3.69,overcast clouds
20,20,Dingle,PH,10.9995,122.6711,73.33,93,5,6.51,clear sky
21,21,Kalmunai,LK,7.4167,81.8167,77.68,84,87,7.07,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='any')
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Hithadhoo,MV,-0.6000,73.0833,82.11,74,25,5.39,scattered clouds
5,5,Saint-Joseph,RE,-21.3667,55.6167,77.36,84,28,8.48,scattered clouds
7,7,Meulaboh,ID,4.1363,96.1285,78.24,84,99,3.42,overcast clouds
8,8,Coihaique,CL,-45.5752,-72.0662,70.41,49,40,11.50,scattered clouds
10,10,Rikitea,PF,-23.1203,-134.9692,78.12,81,100,20.78,overcast clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Hithadhoo,MV,82.11,scattered clouds,-0.6000,73.0833,
5,Saint-Joseph,RE,77.36,scattered clouds,-21.3667,55.6167,
7,Meulaboh,ID,78.24,overcast clouds,4.1363,96.1285,
8,Coihaique,CL,70.41,scattered clouds,-45.5752,-72.0662,
10,Rikitea,PF,78.12,overcast clouds,-23.1203,-134.9692,
11,Vaini,TO,73.56,light intensity shower rain,-21.2000,-175.2000,
14,Butaritari,KI,76.57,very heavy rain,3.0707,172.7902,
15,Sibolga,ID,76.21,overcast clouds,1.7427,98.7792,
20,Dingle,PH,73.33,clear sky,10.9995,122.6711,
21,Kalmunai,LK,77.68,overcast clouds,7.4167,81.8167,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
            
            
hotel_df.head(10)            
        
        

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Hithadhoo,MV,82.11,scattered clouds,-0.6000,73.0833,Scoop Guest House
5,Saint-Joseph,RE,77.36,scattered clouds,-21.3667,55.6167,"""Plantation Bed and Breakfast"
7,Meulaboh,ID,78.24,overcast clouds,4.1363,96.1285,Kost Pria
8,Coihaique,CL,70.41,scattered clouds,-45.5752,-72.0662,Diego De Almagro Coyhaique
10,Rikitea,PF,78.12,overcast clouds,-23.1203,-134.9692,People ThankYou
11,Vaini,TO,73.56,light intensity shower rain,-21.2000,-175.2000,Keleti Beach Resort
14,Butaritari,KI,76.57,very heavy rain,3.0707,172.7902,Isles Sunset Lodge
15,Sibolga,ID,76.21,overcast clouds,1.7427,98.7792,Hotel Syariah CN Darussalam
20,Dingle,PH,73.33,clear sky,10.9995,122.6711,SEGAYA PROPERTY
21,Kalmunai,LK,77.68,overcast clouds,7.4167,81.8167,VS Villa


In [26]:
import numpy as np

# 7. Drop the rows where there is no Hotel Name.

c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df.head()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Hithadhoo,MV,82.11,scattered clouds,-0.6000,73.0833,Scoop Guest House
5,Saint-Joseph,RE,77.36,scattered clouds,-21.3667,55.6167,"""Plantation Bed and Breakfast"
7,Meulaboh,ID,78.24,overcast clouds,4.1363,96.1285,Kost Pria
8,Coihaique,CL,70.41,scattered clouds,-45.5752,-72.0662,Diego De Almagro Coyhaique
10,Rikitea,PF,78.12,overcast clouds,-23.1203,-134.9692,People ThankYou


In [30]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [31]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [32]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))